In [1]:
import numpy as np
import pandas as pd
import nibabel as nib
import matplotlib.pyplot as plt

In [27]:
# Select the base to the datapath
path_database_D = "D:/UNF_data/07_08_2024/T1w/thoracic_volume/thoracic_wo_ts_labels.nii.gz"
path_database_C = "C:/Users/User/msc_project/Image-processing-strategies/mask_control/projects/db0_028/final_test2.nii.gz"


In [28]:
image_gt_data = nib.load(path_database_C)  # labels_total_seg.nii.gz
image_gt = np.array(image_gt_data.get_fdata())
image_gt.shape

(192, 260, 320)

In [30]:
# Output file name choose now
out_fn = "finalissmo_test2.nii.gz"

In [39]:
# Code to force mask to be binarized
# This seems to be an issue when working with TotalSegmentator masks
Size_X=(image_gt.shape)[0]
Size_Y=(image_gt.shape)[1]
Size_Z=(image_gt.shape)[2]
img_new_LE=np.zeros((Size_X,Size_Y,Size_Z))

for i in range(len(image_gt[:,0,0])):
    for j in range( len(image_gt[0,:,0])):
        for k in range( len(image_gt[0,0,:])):
            if image_gt[i,j,z] > 0.5 :
                img_new_LE[i,j,z] = 1

                
new_nifti_img = nib.Nifti1Image(img_new_LE, image_gt_data.affine)
nib.save(new_nifti_img, out_fn)

In [ ]:
# Here you can copy and paste the code above and create your own conditions!


In [8]:
# This to force a value to be the only one
# This happens when we multiply by a negative value with sct

value = 324

Size_X=(image_gt.shape)[0]
Size_Y=(image_gt.shape)[1]
Size_Z=(image_gt.shape)[2]
img_new_LE=np.zeros((Size_X,Size_Y,Size_Z))

for i in range(len(image_gt[:,0,0])):
    for j in range( len(image_gt[0,:,0])):
        for z in range( len(image_gt[0,0,:])):
            if image_gt[i,j,z] > 0.5 :
                img_new_LE[i,j,z] = value
            else:
              img_new_LE[i,j,z] = 0
                
new_nifti_img = nib.Nifti1Image(img_new_LE, image_gt_data.affine)
nib.save(new_nifti_img, out_fn)

*Special thank you to fellow Peruvian, researcher and friend N. Medina - Aix Marseille*

In [21]:
# Sometimes when tresholding the initial anatomical image, the binarized has wholes on the body of the image.
# So we need some way to fill this wholes automatically.

def get_neighbors(volume, x, y, z):
    neighbors = []
    for i in range(-1, 2):
        for j in range(-1, 2):
            for k in range(-1, 2):
                if i == 0 and j == 0 and k == 0:
                    continue
                xi, yj, zk = x + i, y + j, z + k
                if 0 <= xi < volume.shape[0] and 0 <= yj < volume.shape[1] and 0 <= zk < volume.shape[2]:
                    neighbors.append(volume[xi, yj, zk])
    return neighbors

def fill_holes(volume):
    filled_volume = volume.copy()
    for x in range(volume.shape[0]):
        for y in range(volume.shape[1]):
            for z in range(volume.shape[2]):
                if volume[x, y, z] == 0:  # Assuming 0 is the "hole"
                    neighbors = get_neighbors(volume, x, y, z)
                    if sum(neighbors) > len(neighbors) // 2:  # Majority vote
                        filled_volume[x, y, z] = 1
    return filled_volume



In [22]:
# Image to load and fill holes
path_database = "correc_thoracic_db0_028.nii.gz"

wholes_image = nib.load(path_database)  # labels_total_seg.nii.gz
wholes_data = np.array(image_gt_data.get_fdata())
wholes_data.shape

(256, 320, 320)

In [23]:
filled_vol = fill_holes(wholes_data)

In [24]:
# Saving and checking if the wholes were filled correctly
fn = "filled_wholes"
file = "thoracic_db0_028"

out_fn2 = fn + "_" + file + ".nii.gz"
filled_vol_nii = nib.Nifti1Image(filled_vol, affine = wholes_image.affine)
nib.save(filled_vol_nii, out_fn2)

In [30]:
# Using the labels from Total Segmentator to leave and empty space on the place of the mask

# Load the segmentations

segmentation_path = "D:/UNF_data/07_08_2024/T1w/thoracic_volume/thoracic_labels_bin.nii.gz"

segms = nib.load(segmentation_path)
segmentation_data = segms.get_fdata()

dimensions = np.array(segmentation_data.shape)

final_filled_vol = filled_vol.copy()
for i in range(dimensions[0]):
    for j in range(dimensions[1]):
        for k in range(dimensions[2]):
            pixel = segmentation_data[i,j,k]
            if pixel == 1:
                final_filled_vol[i,j,k] = 0

out_fn3 = "final_" + fn + "_" + file + ".nii.gz"

final_nii = nib.Nifti1Image(final_filled_vol, affine = segms.affine)
nib.save(final_nii, out_fn3)